In [1]:
import requests
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [2]:
import pandas as pd
from pandas.io.json import json_normalize
import matplotlib.pyplot as plt
from tqdm import tqdm
import time
import random

In [3]:
auth_url = "https://www.strava.com/oauth/token"
activites_url = "https://www.strava.com/api/v3/athlete/activities"

payload = {
    'client_id': "105916",
    'client_secret': '73483d8cc7dee0aee1d0cb65d734d41f04127934',
    'refresh_token': '7eb238cedd06a1b5bfd06fe09a723fe8101ac40d',
    'grant_type': "refresh_token",
    'f': 'json'
}

print("Requesting Token...\n")
res = requests.post(auth_url, data=payload, verify=False)
access_token = res.json()['access_token']
print("Access Token = {}\n".format(access_token))

print("Requesting pages (200 activities per full page)...")
activities_df = pd.DataFrame()
page = 1
page_non_empty = True
while page_non_empty:
    header = {'Authorization': 'Bearer ' + access_token}
    param = {'per_page': 200, 'page': page}
    my_activities = requests.get(activites_url, headers=header, params=param).json()
    activities_df = activities_df.append(my_activities, ignore_index=True)
    page_non_empty = bool(my_activities)
    print(page)
    page += 1

print("\n", len(activities_df), "activities downloaded")

Requesting Token...

Access Token = d3b92abf5c4ab5c5c052e38936dd33dfe5039a7a

Requesting pages (200 activities per full page)...


/tmp/ipykernel_1006/1686958614.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  activities_df = activities_df.append(my_activities, ignore_index=True)


1


/tmp/ipykernel_1006/1686958614.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  activities_df = activities_df.append(my_activities, ignore_index=True)


2


/tmp/ipykernel_1006/1686958614.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  activities_df = activities_df.append(my_activities, ignore_index=True)


3


/tmp/ipykernel_1006/1686958614.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  activities_df = activities_df.append(my_activities, ignore_index=True)


4
5

 753 activities downloaded


/tmp/ipykernel_1006/1686958614.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  activities_df = activities_df.append(my_activities, ignore_index=True)


In [4]:
activities_df

,resource_state,athlete,name,distance,moving_time,elapsed_time,total_elevation_gain,type,sport_type,workout_type,...,upload_id_str,external_id,from_accepted_tag,pr_count,total_photo_count,has_kudoed,average_cadence,max_watts,weighted_average_watts,average_temp
0,2,"{'id': 20548011, 'resource_state': 1}",Ongekend maar waar,30052.7,3827,4035,89.0,Ride,Ride,10.0,...,9543415572,garmin_ping_269355759672,False,13,0,False,NaN,NaN,NaN,NaN
1,2,"{'id': 20548011, 'resource_state': 1}",Laatste prikje voor Japan,4386.1,1395,2205,11.0,Run,Run,0.0,...,9526583754,garmin_ping_268899401645,False,0,0,False,83.0,NaN,NaN,NaN
2,2,"{'id': 20548011, 'resource_state': 1}",Avondloop,3233.6,1048,1048,4.0,Run,Run,NaN,...,9470838480,garmin_ping_267461010517,False,0,0,False,84.1,NaN,NaN,NaN
3,2,"{'id': 20548011, 'resource_state': 1}",Namiddagloop,3076.0,1071,1079,3.0,Run,Run,NaN,...,9458465860,garmin_ping_267127160507,False,0,0,False,81.1,NaN,NaN,NaN
4,2,"{'id': 20548011, 'resource_state': 1}",Middagloop,2490.5,928,928,2.0,Run,Run,NaN,...,9373763957,garmin_ping_264902790473,False,0,0,False,80.5,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
748,2,"{'id': 20548011, 'resource_state': 1}",Ochtendrit,68912.6,11525,13591,902.5,Ride,Ride,10.0,...,1034693917,3C1F69CB-8898-4483-875B-58644CEBA967,False,5,1,False,NaN,NaN,NaN,NaN
749,2,"{'id': 20548011, 'resource_state': 1}",Middagrit,22880.6,2656,2656,0.0,Ride,Ride,10.0,...,1024624993,FA7C3F63-74AF-48D1-9536-C17CFD277704,False,0,0,False,NaN,NaN,NaN,NaN
750,2,"{'id': 20548011, 'resource_state': 1}",Ochtendrit,31786.2,4729,4729,404.0,Ride,Ride,10.0,...,1024208629,19C5F557-1408-4DC0-B21F-E022B0EC33A9,False,11,0,False,NaN,NaN,NaN,NaN
751,2,"{'id': 20548011, 'resource_state': 1}",🚴🏽💨☀️,36441.3,4297,4297,15.8,Ride,Ride,12.0,...,1010704619,DEDA363F-D666-45A4-A6AB-A6F6281E836A,False,2,0,False,NaN,NaN,NaN,NaN
